In [ ]:
# Install library dependencies (versions fixed to those used at time of publication)
#
%pip install numpy==1.23.5 \
             matplotlib==3.7.1 \
             DeepXDE==1.7.1 \
             tensorflow==2.12.0

In [ ]:
import deepxde as dde
import numpy as np

from pinn import swe_1d
from csv_utils import load_data

# Set random seed to 0 to allow for reproducable randomness in results
#
dde.config.set_random_seed(0)

# Set default float to 32-bit since Tensorflow seems to like that better
# when training on a GPU
#
dde.config.set_default_float('float32')

# Generate domain
#
x_min, x_max = (0, 200)
t_min, t_max = (0, 15)

geom = dde.geometry.Interval(x_min, x_max)
timedomain = dde.geometry.TimeDomain(t_min, t_max)
geomtime = dde.geometry.GeometryXTime(geom, timedomain)

# Point-set boundary condition for physical quantities of interest. In a real-world application, this might 
# correspond to measured data at specific collocation points.
#
observed_measurement_data = load_data(100)  # Load 100 random "measurements"

bc_h_obs = dde.icbc.PointSetBC(observed_measurement_data[:, 0:2], observed_measurement_data[:, 2:3], component=0)
bc_v_obs = dde.icbc.PointSetBC(observed_measurement_data[:, 0:2], observed_measurement_data[:, 3:4], component=1)
bc_alpha_obs = dde.icbc.PointSetBC(observed_measurement_data[:, 0:2], observed_measurement_data[:, 4:5], component=2)

# Periodic boundary conditions for height and velocity, respectively
#
bc_h = dde.icbc.PeriodicBC(geomtime, 0, lambda _, on_boundary: on_boundary, component=0, derivative_order=0)
bc_v = dde.icbc.PeriodicBC(geomtime, 0, lambda _, on_boundary: on_boundary, component=1, derivative_order=0) 

# Periodic boundary conditions for height and velocity derivatives, respectively
#
bc_h_x = dde.icbc.PeriodicBC(geomtime, 0, lambda _, on_boundary: on_boundary, component=0, derivative_order=1)
bc_v_x = dde.icbc.PeriodicBC(geomtime, 0, lambda _, on_boundary: on_boundary, component=1, derivative_order=1)

# Initial condition for wave height
#
ic_h = dde.icbc.IC(
    geomtime,
    lambda x: 2 + np.sin(x[:, 0:1] * np.pi / 100),
    lambda _, on_initial: on_initial,
    component=0,
)

# Initial condition for wave velocity
#
ic_v = dde.icbc.IC(
    geomtime,
    lambda x: 0,
    lambda _, on_initial: on_initial,
    component=1,
)

# Solve system with specified BCs and ICs
#
swe_1d(geomtime, [
    ic_h,
    ic_v
], [
    bc_h, 
    bc_v, 
    bc_h_obs, 
    #bc_v_obs,
    bc_alpha_obs,
    bc_h_x, 
    bc_v_x,
], samples=load_data(1000))

Set the default float type to float32
Compiling model...
Building feed-forward neural network...
'build' took 0.080539 s



2023-04-08 04:24:06.340050: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-08 04:24:06.340298: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-08 04:24:06.340426: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

'compile' took 0.628023 s

Initializing variables...
Training model...

Step      Train loss                                                                                              Test loss                                                                                               Test metric
0         [3.68e-04, 3.12e-01, 3.07e+00, 1.21e+00, 1.32e-01, 1.46e-01, 2.85e+00, 2.01e-02, 6.14e-03, 5.68e-02]    [3.68e-04, 3.12e-01, 3.07e+00, 1.21e+00, 1.32e-01, 1.46e-01, 2.85e+00, 2.01e-02, 6.14e-03, 5.68e-02]    []  
1000      [3.45e-03, 1.37e-02, 2.26e-02, 2.49e-05, 3.41e-03, 2.26e-05, 3.27e-02, 2.64e-05, 1.14e-04, 2.77e-05]    [3.45e-03, 1.37e-02, 2.26e-02, 2.49e-05, 3.41e-03, 2.26e-05, 3.27e-02, 2.64e-05, 1.14e-04, 2.77e-05]    []  
2000      [3.39e-03, 9.47e-03, 7.76e-03, 4.71e-06, 3.51e-03, 4.83e-06, 1.03e-02, 9.85e-06, 2.48e-05, 1.54e-06]    [3.39e-03, 9.47e-03, 7.76e-03, 4.71e-06, 3.51e-03, 4.83e-06, 1.03e-02, 9.85e-06, 2.48e-05, 1.54e-06]    []  
3000      [3.51e-03, 3.73e-02